In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
import random
import operator
import pandas as pd
import numpy as np
import os
import pprint
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.metrics import mean_squared_error as MSE
import pandasql as sqldf

Ziel dieses Notebooks ist es die Hyperparameter Lernrate und Epochenanzahl für das gegebene Neuronale Netz zu optimieren. Hierfür wird eine Klasse verwendet, die in gegebenen Intervallen verschiedene Kombinationen dieser Parameter geordnet ausprobiert (kein random search) und das beste Resultat, basierend auf übergebenen Trainingsdaten zurück gibt. 

In [4]:
def load_data(directory = 'sliced_data'):
    '''
        Funktion, die die aufbereiteten/vorbereiteten Daten aus existierenden CSV Dateien 
        einliest und je nach Vollständigkeit in zwei Dictionaries abspeichert,
        geschlüsselt nach der jeweiligen RaceId
    '''
    #Ids von Rennen, die als Regenrennen identifiziert wurden
    rain_id = [847,861,879,910,914,934,942,953,957,967,950,982]
    if os.path.exists(directory):
        csv_filenames = []
        #auslesen aller csv file dateinamen aus formula 1 datensatz und abspeichern in liste
        for filename in os.listdir(os.getcwd()+'/'+directory):
            typ = filename.split('.')[-1]
            name = filename.split('.')[0]
            if typ == 'csv':
                csv_filenames.append(filename)
        sliced_races = {}
        #einlesen und abspeichern als dataframe aller dateien
        for file in csv_filenames:
            try:
                df = pd.read_csv(directory+'/'+file, engine = 'python', sep = ';', decimal = '.')
                del df['Unnamed: 0']
            except Exception as e:
                df = pd.read_csv(directory+'/'+file, engine = 'c', sep = ';', decimal = '.')
                del df['Unnamed: 0']
                print(e)
            #print(df.head())
            f = int(file.split('_')[-1].split('.')[0])
            df["rain"] = 0
            #bias wird hinzugefügt
            df['bias'] = 1
            #setzen der regenkomponente auf 1 für regenrennen
            if list(df["raceId"])[0] in rain_id:
                df["rain"] = 1
            sliced_races[f] = df
        print('Einlesen der sliced Dateien erfolgreich')
    else:
        raise ('sliced Dateien können nicht eingelesen werden, da kein entsprechendes Verzeichnis existiert!')
        
    if os.path.exists('split_data'):
        csv_filenames = []
        #auslesen aller csv file dateinamen aus formula 1 datensatz und abspeichern in liste
        for filename in os.listdir(os.getcwd()+'/split_data'):
            typ = filename.split('.')[-1]
            name = filename.split('.')[0]
            if typ == 'csv':
                csv_filenames.append(filename)
        split_by_race = {}
        #einlesen und abspeichern als dataframe aller dateien
        for file in csv_filenames:
            try:
                df = pd.read_csv('split_data/'+file, engine = 'python', sep = ';', decimal = '.')
                del df['Unnamed: 0']
            except Exception as e:
                df = pd.read_csv('split_data/'+file, engine = 'c', sep = ';', decimal = '.')
                del df['Unnamed: 0']
                print(e)
            f = int(file.split('_')[-1].split('.')[0])
            #hinzufügen eines bias
            df['bias'] = 1
            split_by_race[f] = df
        print('Einlesen der split Dateien erfolgreich')
    else:
        raise('split Dateien können nicht eingelesen werden, da kein entsprechendes Verzeichnis existiert!')
        
    return sliced_races, split_by_race

def train_test (data_dict, test_num = 5, nogo_columns = []):
    train_data = []
    test_data = []
    test_final = {}
    temp_y_podium = []
    test_races = list(data_dict.keys())
    random.shuffle(test_races)
    test_races = test_races[0:test_num]
    for key, value in data_dict.items():
        helper = key
        for did in value.driverId.unique():
            temp = value.where(value.driverId == did).dropna(how = "all")
            if list(temp["podium_position"])[0] < 0: #Top x finish positions
                pp = 1
            else:
                if key in test_races:
                    temp['sum_milliseconds_pro_lap'] = temp['sum_milliseconds_pro_lap']/60000
                    temp_y = list(temp["podium_position"])#list((temp["total_milliseconds"]/60000))
                    #temp_y = temp_y[0]
                    cols = [col for col in temp.columns if col not in nogo_columns]
                    temp_x = temp[cols]
                    stops=temp_x.sum(axis = 0)[2] #Addierte anzahl an stops bis zu dem Zeitpunkt
                    temp_x = temp_x.tail(1)
                    temp_x['stop_binary'] = stops
                    x_tensor = torch.tensor(temp_x[temp_x.columns].values)
                    #temp_x = x_tensor.float()
                    test_data.append((x_tensor, [temp_y[0]]))
                else:
                    temp['sum_milliseconds_pro_lap'] = temp['sum_milliseconds_pro_lap']/60000
                    temp_y = list(temp["podium_position"])#list((temp["total_milliseconds"]/60000))
                    #temp_y = temp_y[0]
                    cols = [col for col in temp.columns if col not in nogo_columns]
                    temp_x = temp[cols]
                    stops=temp_x.sum(axis = 0)[2] #Addierte anzahl an stops bis zu dem Zeitpunkt
                    temp_x = temp_x.tail(1)
                    temp_x['stop_binary'] = stops
                    x_tensor = torch.tensor(temp_x[temp_x.columns].values)
                    #temp_x = x_tensor.float()
                    train_data.append((x_tensor, [temp_y[0]]))
        if key in test_races:
            test_final[key]=test_data
        test_data = []
    random.shuffle(train_data)
    #random.shuffle(test_data)
    #test_data = train_data[len(train_data)-100:]
    train_data = train_data#[0:len(train_data)-100]
            
            #break
            #for i, row in temp.iterrows():
            
    return train_data, test_final

Einlesen der Dateien als Dataframes, welche in zwei Dictionaries gespeichert werden:<br>
- Attribut directory verweißt auf den Ordnernamen der sliced Dateien, Default ist 'sliced_data'
- für die split_by_race Daten sollte kein separates Directory angegeben werden müssen, da es keinen Unterschied zwischen den Rennen gibt! (Nur bei Slicing: Wie viele Prozent des Rennens möchte ich? ergibt ein anderes Directory Sinn)

In [5]:
sliced_races, split_by_race = load_data(directory = 'sliced_data50')

Einlesen der sliced Dateien erfolgreich
Einlesen der split Dateien erfolgreich


Aufteilen des Datensatzes in einen Trainingsteil und einen Testteil, test enthält ein Dictionary mit Tensoren, die einer RaceId zugeordnet sind, train ist eine Liste von Tensoren. Test_num gibt die Anzahl Rennen an, die als Testdaten zufällig gewählt werden sollen, nogo_columns bezeichnet die Spalten, die nicht beachtet werden sollen, bisher aber im DS enthalten sind

In [6]:
nogo_columns = ['year', 'podium_position', 'raceId','lap_number','total_laps','driverId',
                'grandprix_name', 'driver_fullname',
               'constructor_name', #'total_laps',
               #'status_clean', 'constructorId',
                'total_milliseconds',
               'lap_in_milliseconds']
train, test = train_test(sliced_races, test_num = 5, nogo_columns = nogo_columns)

Definieren von notwendigen Klassen: 
    
Zuerst wird die Klasse für das Neuronale Netz definiert, danach die Klasse für die Optimierung der Hyperparameter Lernrate und Epochen Anzahl

In [10]:
class Netz(nn.Module):
    def __init__(self):
        super(Netz,self).__init__()
        self.fc1 = nn.Linear(111, 150)
        self.fc2 = nn.Linear(150, 180)
        self.fc3 = nn.Linear(180, 190)
        self.fc4 = nn.Linear(190, 120)
        self.fc5 = nn.Linear(120, 100)
        self.fc6 = nn.Linear(100, 70)
        self.fc7 = nn.Linear(70, 30)
        self.fc8 = nn.Linear(30, 1)
        self.dropout = nn.Dropout()
        
        
    def forward(self,x):
        x = self.fc1(x.float())
        x = F.relu(x.float())
        x = self.fc2(x.float())
        x = F.relu(x.float())
        x = self.dropout(x)
        x = self.fc3(x.float())
        x = F.relu(x.float())
        x = self.fc4(x.float())
        x = F.relu(x.float())
        #x = self.dropout(x)
        x = self.fc5(x.float())
        x = F.relu(x.float())
        x = self.fc6(x.float())
        x = F.relu(x.float())
        x = self.fc7(x.float())
        x = F.relu(x.float())
        x = self.fc8(x.float())
        return x
        

In [11]:
class HP_Optimizer(object):
    
    def __init__(self, lr_range = (0.0001,0.0001), step_size = 0.0001, max_epochs = (2,2), opt = 'Adam', cuda = True):
        
        self.__model = Netz()
        self.__lr = lr_range
        self.__epochs = max_epochs
        self.__optimizer = opt
        self.__steps = step_size
        self.__combination_results = {}
        self.__combination_overview = {}
        self.train_data = None
        self.test_data = None
        self.cuda = cuda
        self.opt_combination = {}
        
        
    def validate_combinations(self):
        
        specifics = {}
        if self.__optimizer == 'Adam':
            
            #definieren der range für die lernratenoptimierung
            lr_s = self.__lr[0]
            lr_e = self.__lr[1]
            
            #wurde eine range für die anzahl der epochen übergeben?
            if self.__epochs[0] == self.__epochs[1]:
                #hyperparamter epochenanzahl wird nicht optimiert
                max_epoch = self.__epochs[0]
                
                if lr_s == lr_e:
                    #es ist keine range für die lernrate gegeben, in der diese optimiert werden soll
                    print('Parameter Epochen und Lernrate können nicht optimiert werden, da kein Intervall übergeben wurde')
    
                    #setzen des optimizers als Adam
                    self.__model = Netz()
                    optimizer = optim.Adam(self.__model.parameters(), lr = lr_s)
                
                    for epoch in range(1,max_epoch):
                        self.__train(epoch, optimizer)  
                        
                    result = self.__test()
                    A = result.prediction.tolist()
                    y = result.target.tolist()
                    mae = MAE(A,y)
                    specifics = {}
                    specifics['lr'] = lr_s
                    specifics['epochen'] = max_epoch
                    
                    key = random.randint(0,10000)
                    while key in list(self.__combination_results.keys()):
                         key = random.randint(0,10000)
                    
                    self.__combination_results[key] = mae
                    self.__combination_overview[key] = specifics
                    
                            
                else:
                
                    for l in np.arange(lr_s,lr_e, self.__steps):
                
                        #setzen des optimizers als Adam
                        self.__model = Netz()
                        optimizer = optim.Adam(self.__model.parameters(), lr = l)
                        #trainieren des modells
                        for epoch in range(1,max_epoch):
                            self.__train(epoch, optimizer)  
                        
                        result = self.__test()
                        A = result.prediction.tolist()
                        y = result.target.tolist()
                        mae = MAE(A,y)
                        specifics = {}
                        specifics['lr'] = l
                        specifics['epochen'] = max_epoch
                        
                        #abspeichern der gewonnenen informationen (MAE nach lr und anzahl durchgeführter epochen)
                        key = random.randint(0,10000)
                        while key in list(self.__combination_results.keys()):
                             key = random.randint(0,10000)
                        
                        self.__combination_results[key] = mae
                        self.__combination_overview[key] = specifics
                    
                    
            else:
                for max_epoch in range(self.__epochs[0], self.__epochs[1]):
                    #definieren der range für die lernratenoptimierung
                    lr_s = self.__lr[0]
                    lr_e = self.__lr[1]
            
                    #hyperparamter epochenanzahl wird nicht optimiert
                        
                    if lr_s == lr_e:
                        #es ist keine range für die lernrate gegeben, in der diese optimiert werden soll
                        #print('Parameter Epochen und Lernrate können nicht optimiert werden, da kein Intervall übergeben wurde')
            
                        #setzen des optimizers als Adam
                        self.__model = Netz()
                        optimizer = optim.Adam(self.__model.parameters(), lr = lr_s)
                        
                        for epoch in range(1,max_epoch):
                            self.__train(epoch, optimizer)  
                                
                        result = self.__test()
                        A = result.prediction.tolist()
                        y = result.target.tolist()
                        mae = MAE(A,y)
                        specifics = {}
                        specifics['lr'] = lr_s
                        specifics['epochen'] = max_epoch
                          
                        key = random.randint(0,10000)
                        while key in list(self.__combination_results.keys()):
                             key = random.randint(0,10000)
                            
                        self.__combination_results[key] = mae
                        self.__combination_overview[key] = specifics
                            
                                    
                    else:
                        
                        for l in np.arange(lr_s,lr_e, self.__steps):
                        
                            #setzen des optimizers als Adam
                            self.__model = Netz()
                            optimizer = optim.Adam(self.__model.parameters(), lr = l)
                            #trainieren des modells
                            for epoch in range(1,max_epoch):
                                self.__train(epoch, optimizer)  
                            
                            result = self.__test()
                            A = result.prediction.tolist()
                            y = result.target.tolist()
                            mae = MAE(A,y)
                            specifics = {}
                            specifics['lr'] = l
                            specifics['epochen'] = max_epoch
                            
                            #abspeichern der gewonnenen informationen (MAE nach lr und anzahl durchgeführter epochen)
                            key = random.randint(0,10000)
                            while key in list(self.__combination_results.keys()):
                                 key = random.randint(0,10000)
                            
                            self.__combination_results[key] = mae
                            self.__combination_overview[key] = specifics
            
            #finden der besten kombination nach minimalstem Error (MAE)
            key_min = min(self.__combination_results.keys(), key=(lambda k: self.__combination_results[k]))
            best_combination = self.__combination_overview[key_min]
            best_combination['mae'] = self.__combination_results[key_min]
            self.opt_combination = best_combination
        else:
            raise ('No valid optimizer given! Try Adam for example!')
            
    def __train(self, epoch, optimizer):
        if self.cuda:
            self.__model.cuda()
            self.__model.train()
            batch_id = 0
            for data, target in self.train_data:
                data = data.cuda()
                target = torch.Tensor(target).unsqueeze(0).cuda()
                shape = target.size()[1]
                target = target.resize(shape,1).cuda()
                optimizer.zero_grad()
                out = self.__model(data)
                #print("Out: ", out, out.size())
                #print("Target: ", target, target.size())
                criterion = nn.MSELoss()
                loss = criterion(out, target)
                loss.backward()
                optimizer.step()
                print("Train Epoche: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                    epoch, batch_id *len(data), len(train_data),
                100. * batch_id / len(train_data), loss.item()))
                batch_id +=1
        else:
            self.__model.train()
            batch_id = 0
            for data, target in self.train_data:
                #data = data.cuda()
                target = torch.Tensor(target).unsqueeze(0)#.cuda()
                shape = target.size()[1]
                target = target.resize(shape,1)#.cuda()
                optimizer.zero_grad()
                out = self.__model(data)
                #print("Out: ", out, out.size())
                #print("Target: ", target, target.size())
                criterion = nn.MSELoss()
                loss = criterion(out, target)
                loss.backward()
                optimizer.step()
                #print("Train Epoche: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                #    epoch, batch_id *len(data), len(train_data),
                #100. * batch_id / len(train_data), loss.item()))
                batch_id +=1
            
    def __test(self):
        total = 0
        count = 0
        result_dict = {}
        result = pd.DataFrame(columns = ['target','prediction'])
        help_dict = {}
        if self.cuda:
            self.__model.cuda()
            for key in self.test_data.keys():
                #print(key)
                help_dict = {}
                for data, target in self.test_data[key]:
                    self.__model.eval()
                    #files.listdir(path)
                    data = data.cuda()
                    target = torch.Tensor(target).unsqueeze(0)
                    shape = target.size()[1]
                    target = target.resize(shape,1).cuda()
                    out = self.__model(data).cpu()
                    #print(out)
                    out = out.detach().numpy()
                    #out = np.round(out)
                    target = target.cpu()
                    target = target.detach().numpy()
                    #print(data)
                    #print(data["driverId"])
                    total += abs(out - target[0][0])
                    #print("current_position: ", data[0][0].item())
                    #print("Output: ", out)
                    #print("Target: ", target)
                    help_dict[target[0][0]] = out
                    #print("Difference: ", out - target)
                    count+=1
                #Auslesen der predicteten Werte A und der zugehörigen targets y
                A = [x[0][0] for x in list((help_dict.values()))]
                y = list(help_dict.keys())
                
                #Anfügen der Werte an Result
                t = pd.DataFrame()
                t['target'] = y
                t['prediction_value'] = A
                t = sqldf.sqldf('''select * from t order by prediction_value ASC''')
                t.reset_index(inplace = True)
                t.rename(columns = {'index':'prediction'}, inplace = True)
                t['prediction'] = t['prediction']+1
                
                
                result = result.append(t)
                
        else:
            for raceId in self.test_data.keys():
                #print(key)
                help_dict = {}
                for data, target in self.test_data[raceId]:
                    self.__model.eval()
                    #files.listdir(path)
                    #data = data.cuda()
                    target = torch.Tensor(target).unsqueeze(0)
                    shape = target.size()[1]
                    target = target.resize(shape,1)#.cuda()
                    out = self.__model(data)#.cpu()
                    #print(out)
                    out = out.detach().numpy()
                    #out = np.round(out)
                    #target = target.cpu()
                    target = target.detach().numpy()
                    #print(data)
                    #print(data["driverId"])
                    total += abs(out - target[0][0])
                    #print("current_position: ", data[0][0].item())
                    #print("Output: ", out)
                    #print("Target: ", target)
                    help_dict[target[0][0]] = out
                    #print("Difference: ", out - target)
                    count+=1
                #Auslesen der predicteten Werte A und der zugehörigen targets y
                A = [x[0][0] for x in list((help_dict.values()))]
                y = list(help_dict.keys())
                
                #Anfügen der Werte an Result
                t = pd.DataFrame()
                t['target'] = y
                t['prediction_value'] = A
                t = sqldf.sqldf('''select * from t order by prediction_value ASC''')
                t.reset_index(inplace = True)
                t.rename(columns = {'index':'prediction'}, inplace = True)
                t['prediction'] = t['prediction']+1
                
                result = result.append(t)
        return result
         
            
    def get_all_information(self):
        
        print('Chosen Model:',self.__model)
        print('Learningrate Range:',self.__lr)
        print('Maximum Epochs:', self.__epochs)
        print('Chosen Optimizer:', self.__optimizer)
        print('Result Encoding:', self.__combination_overview)
        print('Results:', self.__combination_results)
        print('Optimale Kombination:', self.opt_combination)
        
    def help(self):
        print('Parameters with defaults:\nlr_range --> (0.0001,0.0001),\nstep_size--> 0.0001,\nmax_epochs-->(2,2),\nopt-->"Adam",\ncuda=True')
        print('lr_range: Tupel with learnrate range')
        print('step_size: float/int for step_size of learnrate')
        print('max_epochs: Tupel with number of epochs range')
        print('opt: Optimizer (by default Adam)')
        print('cuda: True/False if cuda should be used, default = True\n')
        print('Attributes:')
        print('set self.train_data as dictionary with races (form: {raceId: race(dataframe)})')
        print('set self.test_data as dictionary with races (form: {raceId: race(dataframe)})')
        print('self.opt_combination: Dictionary which contains the best combination of the given parameters\n')
        print('Methods:')
        print('call self.validate_combination() to compare all combinations')
        print('get all information/results with self.get_all_information()')

Aufrufen des Optimierers, übergeben der zuvor erzeugten Test- und Trainingsdatensätze, angeben der Intervalle für die die Hyperparameter getestet werden sollen (Lernrate und Epochenanzahl). 

In [13]:
h = HP_Optimizer(lr_range = (0.00001,0.00005),step_size = 0.00001, max_epochs=(1,5),cuda = False)
h.help()
h.train_data = train
h.test_data = test
h.validate_combinations()
h.get_all_information()

Parameters with defaults:
lr_range --> (0.0001,0.0001),
step_size--> 0.0001,
max_epochs-->(2,2),
opt-->"Adam",
cuda=True
lr_range: Tupel with learnrate range
step_size: float/int for step_size of learnrate
max_epochs: Tupel with number of epochs range
opt: Optimizer (by default Adam)
cuda: True/False if cuda should be used, default = True

Attributes:
set self.train_data as dictionary with races (form: {raceId: race(dataframe)})
set self.test_data as dictionary with races (form: {raceId: race(dataframe)})
self.opt_combination: Dictionary which contains the best combination of the given parameters

Methods:
call self.validate_combination() to compare all combinations
get all information/results with self.get_all_information()


c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\pandas\core\frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,
c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\torch\tensor.py:362: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\pandas\core\frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,
c:\users\hwebe\appdata\local\programs\python\python37\lib\site-pa

c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\pandas\core\frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,
c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\torch\tensor.py:362: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\pandas\core\frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,
c:\users\hwebe\appdata\local\programs\python\python37\lib\site-pa

c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\pandas\core\frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,
c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\torch\tensor.py:362: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\pandas\core\frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,
c:\users\hwebe\appdata\local\programs\python\python37\lib\site-pa

c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\pandas\core\frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,
c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\torch\tensor.py:362: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\pandas\core\frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,
c:\users\hwebe\appdata\local\programs\python\python37\lib\site-pa

c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\torch\tensor.py:362: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\pandas\core\frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,
c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\torch\tensor.py:362: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\pandas\core\frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the

Chosen Model: Netz(
  (fc1): Linear(in_features=111, out_features=150, bias=True)
  (fc2): Linear(in_features=150, out_features=180, bias=True)
  (fc3): Linear(in_features=180, out_features=190, bias=True)
  (fc4): Linear(in_features=190, out_features=120, bias=True)
  (fc5): Linear(in_features=120, out_features=100, bias=True)
  (fc6): Linear(in_features=100, out_features=70, bias=True)
  (fc7): Linear(in_features=70, out_features=30, bias=True)
  (fc8): Linear(in_features=30, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)
Learningrate Range: (1e-05, 5e-05)
Maximum Epochs: (1, 5)
Chosen Optimizer: Adam
Result Encoding: {417: {'lr': 1e-05, 'epochen': 1}, 5057: {'lr': 2e-05, 'epochen': 1}, 6316: {'lr': 3.0000000000000004e-05, 'epochen': 1}, 415: {'lr': 4e-05, 'epochen': 1}, 5528: {'lr': 1e-05, 'epochen': 2}, 2417: {'lr': 2e-05, 'epochen': 2}, 3491: {'lr': 3.0000000000000004e-05, 'epochen': 2}, 6646: {'lr': 4e-05, 'epochen': 2}, 6029: {'lr': 1e-05, 'epochen': 3},

c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\torch\tensor.py:362: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\pandas\core\frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


Es werden die als optimal in den gegebenen Intervallen genommenen Werte aus dem Optimierer genommen und ein neues Modell mit diesen Parametern trainiert. Zum Schluss werden die Ergebnisse des Testlaufes auf den Testdaten ausgegeben.

In [14]:
max_epochs = h.opt_combination['epochen']
lr = h.opt_combination['lr']

cuda = input('Cuda? [y/n]: ')
model = Netz()
if cuda.lower() == 'y':
    model.cuda() 

    
optimizer = optim.Adam(model.parameters(), lr = lr)     

def train_cuda(epoch):
    model.train()
    batch_id = 0
    for data, target in train:
        #data = data.cuda()
        target = torch.Tensor(target).unsqueeze(0).cuda()
        shape = target.size()[1]
        target = target.resize(shape,1).cuda()
        optimizer.zero_grad()
        out = model(data)
        #print("Out: ", out, out.size())
        #print("Target: ", target, target.size())
        criterion = nn.MSELoss()
        loss = criterion(out, target)
        loss.backward()
        optimizer.step()
        print("Train Epoche: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
            epoch, batch_id *len(data), len(train),
        100. * batch_id / len(train), loss.item()))
        batch_id +=1
        
    #random.shuffle(train_data)
def train_(epoch):
    model.train()
    batch_id = 0
    for data, target in train:
        #data = data.cuda()
        target = torch.Tensor(target).unsqueeze(0)#.cuda()
        shape = target.size()[1]
        target = target.resize(shape,1)#.cuda()
        optimizer.zero_grad()
        out = model(data)
        #print("Out: ", out, out.size())
        #print("Target: ", target, target.size())
        criterion = nn.MSELoss()
        loss = criterion(out, target)
        loss.backward()
        optimizer.step()
        print("Train Epoche: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
            epoch, batch_id *len(data), len(train),
        100. * batch_id / len(train), loss.item()))
        batch_id +=1
        
    #random.shuffle(train_data)
if cuda.lower() == 'y':
    for epoch in range(1,max_epochs):
        train_cuda(epoch) 
else:
    for epoch in range(1,max_epochs):
        train_(epoch)  
        

Cuda? [y/n]: n


c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\torch\tensor.py:362: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


Train Epoche: 1 [0/2790 (0%)]	Loss: 16.623455
Train Epoche: 1 [1/2790 (0%)]	Loss: 9.453110
Train Epoche: 1 [2/2790 (0%)]	Loss: 227.452362
Train Epoche: 1 [3/2790 (0%)]	Loss: 198.130554
Train Epoche: 1 [4/2790 (0%)]	Loss: 258.298279
Train Epoche: 1 [5/2790 (0%)]	Loss: 101.510941
Train Epoche: 1 [6/2790 (0%)]	Loss: 82.247772
Train Epoche: 1 [7/2790 (0%)]	Loss: 50.051319
Train Epoche: 1 [8/2790 (0%)]	Loss: 1.160154
Train Epoche: 1 [9/2790 (0%)]	Loss: 1.156068
Train Epoche: 1 [10/2790 (0%)]	Loss: 579.540466
Train Epoche: 1 [11/2790 (0%)]	Loss: 16.601299
Train Epoche: 1 [12/2790 (0%)]	Loss: 82.275238
Train Epoche: 1 [13/2790 (0%)]	Loss: 50.084854
Train Epoche: 1 [14/2790 (1%)]	Loss: 101.447350
Train Epoche: 1 [15/2790 (1%)]	Loss: 36.851448
Train Epoche: 1 [16/2790 (1%)]	Loss: 227.071609
Train Epoche: 1 [17/2790 (1%)]	Loss: 65.189438
Train Epoche: 1 [18/2790 (1%)]	Loss: 258.247162
Train Epoche: 1 [19/2790 (1%)]	Loss: 291.234619
Train Epoche: 1 [20/2790 (1%)]	Loss: 25.665932
Train Epoche: 1 [

Train Epoche: 1 [171/2790 (6%)]	Loss: 474.806305
Train Epoche: 1 [172/2790 (6%)]	Loss: 482.277588
Train Epoche: 1 [173/2790 (6%)]	Loss: 189.346024
Train Epoche: 1 [174/2790 (6%)]	Loss: 567.160400
Train Epoche: 1 [175/2790 (6%)]	Loss: 163.277649
Train Epoche: 1 [176/2790 (6%)]	Loss: 312.655243
Train Epoche: 1 [177/2790 (6%)]	Loss: 8.000820
Train Epoche: 1 [178/2790 (6%)]	Loss: 247.708389
Train Epoche: 1 [179/2790 (6%)]	Loss: 482.471527
Train Epoche: 1 [180/2790 (6%)]	Loss: 12.984749
Train Epoche: 1 [181/2790 (6%)]	Loss: 474.668579
Train Epoche: 1 [182/2790 (7%)]	Loss: 23.588753
Train Epoche: 1 [183/2790 (7%)]	Loss: 75.337372
Train Epoche: 1 [184/2790 (7%)]	Loss: 6.913579
Train Epoche: 1 [185/2790 (7%)]	Loss: 422.346985
Train Epoche: 1 [186/2790 (7%)]	Loss: 115.152885
Train Epoche: 1 [187/2790 (7%)]	Loss: 21.527651
Train Epoche: 1 [188/2790 (7%)]	Loss: 187.153656
Train Epoche: 1 [189/2790 (7%)]	Loss: 44.144505
Train Epoche: 1 [190/2790 (7%)]	Loss: 310.158020
Train Epoche: 1 [191/2790 (7%

Train Epoche: 1 [367/2790 (13%)]	Loss: 29.843382
Train Epoche: 1 [368/2790 (13%)]	Loss: 45.554146
Train Epoche: 1 [369/2790 (13%)]	Loss: 4.575868
Train Epoche: 1 [370/2790 (13%)]	Loss: 0.146987
Train Epoche: 1 [371/2790 (13%)]	Loss: 0.489115
Train Epoche: 1 [372/2790 (13%)]	Loss: 40.154171
Train Epoche: 1 [373/2790 (13%)]	Loss: 43.393379
Train Epoche: 1 [374/2790 (13%)]	Loss: 23.444567
Train Epoche: 1 [375/2790 (13%)]	Loss: 245.022263
Train Epoche: 1 [376/2790 (13%)]	Loss: 24.573378
Train Epoche: 1 [377/2790 (14%)]	Loss: 0.987408
Train Epoche: 1 [378/2790 (14%)]	Loss: 17.666965
Train Epoche: 1 [379/2790 (14%)]	Loss: 68.907593
Train Epoche: 1 [380/2790 (14%)]	Loss: 57.182854
Train Epoche: 1 [381/2790 (14%)]	Loss: 33.502525
Train Epoche: 1 [382/2790 (14%)]	Loss: 2.588510
Train Epoche: 1 [383/2790 (14%)]	Loss: 4.181066
Train Epoche: 1 [384/2790 (14%)]	Loss: 25.880713
Train Epoche: 1 [385/2790 (14%)]	Loss: 48.913094
Train Epoche: 1 [386/2790 (14%)]	Loss: 223.321320
Train Epoche: 1 [387/279

...

Train Epoche: 3 [2025/2790 (73%)]	Loss: 0.584701
Train Epoche: 3 [2026/2790 (73%)]	Loss: 1.033808
Train Epoche: 3 [2027/2790 (73%)]	Loss: 0.167820
Train Epoche: 3 [2028/2790 (73%)]	Loss: 1.621973
Train Epoche: 3 [2029/2790 (73%)]	Loss: 1.393851
Train Epoche: 3 [2030/2790 (73%)]	Loss: 11.867274
Train Epoche: 3 [2031/2790 (73%)]	Loss: 0.939467
Train Epoche: 3 [2032/2790 (73%)]	Loss: 9.023584
Train Epoche: 3 [2033/2790 (73%)]	Loss: 10.753057
Train Epoche: 3 [2034/2790 (73%)]	Loss: 3.659949
Train Epoche: 3 [2035/2790 (73%)]	Loss: 2.218213
Train Epoche: 3 [2036/2790 (73%)]	Loss: 6.234208
Train Epoche: 3 [2037/2790 (73%)]	Loss: 8.902089
Train Epoche: 3 [2038/2790 (73%)]	Loss: 3.712308
Train Epoche: 3 [2039/2790 (73%)]	Loss: 160.047531
Train Epoche: 3 [2040/2790 (73%)]	Loss: 3.864165
Train Epoche: 3 [2041/2790 (73%)]	Loss: 38.672318
Train Epoche: 3 [2042/2790 (73%)]	Loss: 0.309829
Train Epoche: 3 [2043/2790 (73%)]	Loss: 0.047348
Train Epoche: 3 [2044/2790 (73%)]	Loss: 0.362343
Train Epoche: 3

Train Epoche: 3 [2197/2790 (79%)]	Loss: 0.008983
Train Epoche: 3 [2198/2790 (79%)]	Loss: 7.520207
Train Epoche: 3 [2199/2790 (79%)]	Loss: 3.072331
Train Epoche: 3 [2200/2790 (79%)]	Loss: 13.229330
Train Epoche: 3 [2201/2790 (79%)]	Loss: 4.536894
Train Epoche: 3 [2202/2790 (79%)]	Loss: 0.494736
Train Epoche: 3 [2203/2790 (79%)]	Loss: 0.323600
Train Epoche: 3 [2204/2790 (79%)]	Loss: 0.096739
Train Epoche: 3 [2205/2790 (79%)]	Loss: 1.015034
Train Epoche: 3 [2206/2790 (79%)]	Loss: 7.889688
Train Epoche: 3 [2207/2790 (79%)]	Loss: 195.726257
Train Epoche: 3 [2208/2790 (79%)]	Loss: 3.036866
Train Epoche: 3 [2209/2790 (79%)]	Loss: 100.951866
Train Epoche: 3 [2210/2790 (79%)]	Loss: 0.781329
Train Epoche: 3 [2211/2790 (79%)]	Loss: 334.452423
Train Epoche: 3 [2212/2790 (79%)]	Loss: 1.781657
Train Epoche: 3 [2213/2790 (79%)]	Loss: 3.995894
Train Epoche: 3 [2214/2790 (79%)]	Loss: 4.105513
Train Epoche: 3 [2215/2790 (79%)]	Loss: 27.804417
Train Epoche: 3 [2216/2790 (79%)]	Loss: 17.115705
Train Epoch

Train Epoche: 3 [2373/2790 (85%)]	Loss: 10.555763
Train Epoche: 3 [2374/2790 (85%)]	Loss: 5.814373
Train Epoche: 3 [2375/2790 (85%)]	Loss: 0.729349
Train Epoche: 3 [2376/2790 (85%)]	Loss: 8.126824
Train Epoche: 3 [2377/2790 (85%)]	Loss: 0.132383
Train Epoche: 3 [2378/2790 (85%)]	Loss: 1.130662
Train Epoche: 3 [2379/2790 (85%)]	Loss: 7.045170
Train Epoche: 3 [2380/2790 (85%)]	Loss: 7.693352
Train Epoche: 3 [2381/2790 (85%)]	Loss: 3.244709
Train Epoche: 3 [2382/2790 (85%)]	Loss: 2.044923
Train Epoche: 3 [2383/2790 (85%)]	Loss: 14.317619
Train Epoche: 3 [2384/2790 (85%)]	Loss: 0.145124
Train Epoche: 3 [2385/2790 (85%)]	Loss: 29.956751
Train Epoche: 3 [2386/2790 (86%)]	Loss: 25.804567
Train Epoche: 3 [2387/2790 (86%)]	Loss: 2.754776
Train Epoche: 3 [2388/2790 (86%)]	Loss: 0.008519
Train Epoche: 3 [2389/2790 (86%)]	Loss: 18.154232
Train Epoche: 3 [2390/2790 (86%)]	Loss: 5.883422
Train Epoche: 3 [2391/2790 (86%)]	Loss: 0.464496
Train Epoche: 3 [2392/2790 (86%)]	Loss: 3.583368
Train Epoche: 3

Train Epoche: 3 [2552/2790 (91%)]	Loss: 10.372173
Train Epoche: 3 [2553/2790 (92%)]	Loss: 1.694474
Train Epoche: 3 [2554/2790 (92%)]	Loss: 65.382324
Train Epoche: 3 [2555/2790 (92%)]	Loss: 1.306986
Train Epoche: 3 [2556/2790 (92%)]	Loss: 2.820485
Train Epoche: 3 [2557/2790 (92%)]	Loss: 4.283683
Train Epoche: 3 [2558/2790 (92%)]	Loss: 0.066559
Train Epoche: 3 [2559/2790 (92%)]	Loss: 0.847490
Train Epoche: 3 [2560/2790 (92%)]	Loss: 14.431037
Train Epoche: 3 [2561/2790 (92%)]	Loss: 4.390597
Train Epoche: 3 [2562/2790 (92%)]	Loss: 1.388897
Train Epoche: 3 [2563/2790 (92%)]	Loss: 6.683902
Train Epoche: 3 [2564/2790 (92%)]	Loss: 197.029297
Train Epoche: 3 [2565/2790 (92%)]	Loss: 5.057154
Train Epoche: 3 [2566/2790 (92%)]	Loss: 11.034471
Train Epoche: 3 [2567/2790 (92%)]	Loss: 11.936596
Train Epoche: 3 [2568/2790 (92%)]	Loss: 8.433877
Train Epoche: 3 [2569/2790 (92%)]	Loss: 3.738102
Train Epoche: 3 [2570/2790 (92%)]	Loss: 1.836551
Train Epoche: 3 [2571/2790 (92%)]	Loss: 37.589039
Train Epoche

Train Epoche: 3 [2742/2790 (98%)]	Loss: 8.001323
Train Epoche: 3 [2743/2790 (98%)]	Loss: 0.310706
Train Epoche: 3 [2744/2790 (98%)]	Loss: 4.148894
Train Epoche: 3 [2745/2790 (98%)]	Loss: 4.642902
Train Epoche: 3 [2746/2790 (98%)]	Loss: 4.178305
Train Epoche: 3 [2747/2790 (98%)]	Loss: 3.920235
Train Epoche: 3 [2748/2790 (98%)]	Loss: 0.790910
Train Epoche: 3 [2749/2790 (99%)]	Loss: 4.417399
Train Epoche: 3 [2750/2790 (99%)]	Loss: 25.431044
Train Epoche: 3 [2751/2790 (99%)]	Loss: 8.319139
Train Epoche: 3 [2752/2790 (99%)]	Loss: 5.206734
Train Epoche: 3 [2753/2790 (99%)]	Loss: 8.811007
Train Epoche: 3 [2754/2790 (99%)]	Loss: 7.785920
Train Epoche: 3 [2755/2790 (99%)]	Loss: 0.130330
Train Epoche: 3 [2756/2790 (99%)]	Loss: 7.990509
Train Epoche: 3 [2757/2790 (99%)]	Loss: 5.945649
Train Epoche: 3 [2758/2790 (99%)]	Loss: 36.308598
Train Epoche: 3 [2759/2790 (99%)]	Loss: 223.205063
Train Epoche: 3 [2760/2790 (99%)]	Loss: 0.411709
Train Epoche: 3 [2761/2790 (99%)]	Loss: 2.787699
Train Epoche: 3 

In [15]:
def test_times_cuda(test_final):
    total = 0
    count = 0
    result_dict = {}
    help_dict = {}
    for key in test_final.keys():
        #print(key)
        help_dict = {}
        for data, target in test_final[key]:
            model.eval()
            #files.listdir(path)
            data = data.cuda()
            target = torch.Tensor(target).unsqueeze(0)
            shape = target.size()[1]
            target = target.resize(shape,1).cuda()
            out = model(data).cpu()
            #print(out)
            out = out.detach().numpy()
            #out = np.round(out)
            target = target.cpu()
            target = target.detach().numpy()
            #print(data)
            #print(data["driverId"])
            total += abs(out - target[0][0])
            #print("current_position: ", data[0][0].item())
            #print("Output: ", out)
            #print("Target: ", target)
            help_dict[target[0][0]] = out
            #print("Difference: ", out - target)
            count+=1
        #help_dict = sorted(help_dict.items(), key=operator.itemgetter(1))
        result_dict[key] = help_dict
        
    return result_dict

def test_times(test_final):
    total = 0
    count = 0
    result_dict = {}
    help_dict = {}
    for key in test_final.keys():
        #print(key)
        help_dict = {}
        for data, target in test_final[key]:
            model.eval()
            #files.listdir(path)
            #data = data.cuda()
            target = torch.Tensor(target).unsqueeze(0)
            shape = target.size()[1]
            target = target.resize(shape,1)#.cuda()
            out = model(data)#.cpu()
            #print(out)
            out = out.detach().numpy()
            #out = np.round(out)
            #target = target.cpu()
            target = target.detach().numpy()
            #print(data)
            #print(data["driverId"])
            total += abs(out - target[0][0])
            #print("current_position: ", data[0][0].item())
            #print("Output: ", out)
            #print("Target: ", target)
            help_dict[target[0][0]] = out
            #print("Difference: ", out - target)
            count+=1
        #help_dict = sorted(help_dict.items(), key=operator.itemgetter(1))
        result_dict[key] = help_dict
        
    return result_dict

if cuda.lower() =='y':
    total_results = test_times_cuda(test)
else:
    total_results = test_times(test)


In [19]:
result_dfs = {}
for raceId, dict in total_results.items():    
    #Auswerten der Vorhersagen aus den Outputdaten des Modells
    A = [x[0][0] for x in list((dict.values()))]
    y = list(dict.keys())
    t = pd.DataFrame(columns = ['target', 'prediction'])
    t['target'] = y
    t['prediction'] = A
    #sortieren des DataFrames nach den vorhergesagten Positionen, aufsteigend
    end = sqldf.sqldf('''select * from t order by prediction ASC''')
    end.reset_index(inplace = True)
    #Da DF nun nach prediction aufsteigend sortiert ist, kann veränderter Index als predictete Position gesetzt werden
    end.rename(columns = {'index': 'predicted_position'},inplace = True)
    #zur Übersichtlichkeit wird nun der endgültige DF nach den richtigen Positionen (target) sortiert (aufsteigend)
    end = sqldf.sqldf('''select * from end order by target ASC''')
    end['predicted_position'] = end['predicted_position']+1
    #umstellen der Spaltenreihenfolge
    end = end[['target', 'predicted_position', 'prediction']]
    result_dfs[raceId] = end

In [20]:
for key, value in result_dfs.items():
    print('RaceId:',key)
    print(value,'\n')

RaceId: 852
    target  predicted_position  prediction
0      1.0                   1    1.794732
1      2.0                   2    2.883790
2      3.0                   7    7.683998
3      4.0                   3    3.230210
4      5.0                   9    9.997762
5      6.0                   4    3.871595
6      7.0                   6    7.010408
7      8.0                   5    4.295087
8      9.0                   8    8.238929
9     10.0                  13   12.445101
10    11.0                  11   11.902659
11    12.0                  12   12.144609
12    13.0                  14   12.523632
13    14.0                  16   15.820935
14    15.0                  15   15.770011
15    16.0                  10   11.520939
16    17.0                  17   17.298086
17    18.0                  18   18.589184
18    19.0                  19   18.904297
19    24.0                  20   21.121977 

RaceId: 913
    target  predicted_position  prediction
0      1.0                  

### Anhang

nogo_columns_hannah_kacke = [#'grid',
                #'race_completion',
                'lap_position','circuitId','lap_number',
                'podium_position', 'raceId',
                'grandprix_name', 'driver_fullname',
               'constructor_name', 'total_laps',
               'status_clean', 'constructorId',
                'total_milliseconds', 'driverId'
               'lap_in_milliseconds','year', 'stop_binary','constructorId_1.0',
                 'constructorId_3.0',
                 'constructorId_4.0',
                 'constructorId_5.0',
                 'constructorId_6.0',
                 'constructorId_9.0',
                 'constructorId_10.0',
                 'constructorId_15.0',
                 'constructorId_131.0',
                 'constructorId_164.0',
                 'constructorId_166.0',
                 'constructorId_205.0',
                 'constructorId_206.0',
                 'constructorId_207.0',
                 'constructorId_208.0',
                 'constructorId_209.0',
                 'constructorId_210.0']